In [1]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

import re 

# Tous les films sur la page AlloCiné

**Aucune information dans cette partie**, il faut juste *exécuter les cellules*

> Si par curiosité tu veux avoir un *fichier Excel avec tous les films à l'affiche en ce moment*, tu peux enlever le hastag de la cellule de la partie 1.1 et exécuter la cellule

In [2]:
# Extraire le nombre de page à scraper

url = f"https://www.allocine.fr/film/aucinema/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

all_pages = soup.find_all('div', class_="pagination-item-holder")[0]
nombre_pages = int(str(all_pages)[-12:][:2])

In [3]:
# Scraper tous les films à l'affiche sur AlloCiné

movies_dict = { 'Title': [], 'Code': []}

for i in range(1, nombre_pages+1):
    url = f"https://www.allocine.fr/film/aucinema/?page={i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    all_movies_i = soup.find_all('div', class_="card entity-card entity-card-list cf")
    
    for movie in all_movies_i:
        movie_info = movie.find('h2').find('a')
        movie_class = str(movie_info.attrs['class'][0])
        movie_href = str(movie_info.attrs['href']) 
        movie_title = str(movie_info).replace(movie_href,"").replace(movie_class,"").replace('class=',"").replace('href=',"")
        movie_title = re.sub("\"|\>|\<|\/","",movie_title)[1:-1].strip()
        movie_code = re.findall('\d', movie_href) 
        movie_code = int("".join(movie_code))
        movies_dict["Title"].append(movie_title)
        movies_dict["Code"].append(movie_code)   
        
films_allociné = pd.DataFrame(movies_dict)
films_allociné['Title'] = films_allociné['Title'].apply(lambda x: x.capitalize())
films_allociné.head()

,Title,Code
0,Les trois mousquetaires: d'artagnan,288402
1,"Super mario bros, le film",234099
2,La vie pour de vrai,297442
3,10 jours encore sans maman,298509
4,Donjons &amp; dragons : l'honneur des voleurs,221359


## Exporter tous les films à l'affiche

> **La cellule en-dessous permet d'exporter tous les films à l'affiche au cinéma en ce moment**

In [4]:
# Enlever le hastag et exécuter la cellule si besoin

#films_allociné.to_excel("Films à l'affiche.xlsx") 

# Films de la Liste à l'affiche 

In [5]:
#Importer fichier films Adrien

films_adrien = pd.read_excel('Liste-Films-Charlotte.xlsx')
films_adrien.rename(columns={'Nom':'Title'}, inplace=True)
films_adrien['Title'] = films_adrien['Title'].apply(lambda x: x.capitalize())
films_adrien

,Title
0,101 reykjavik
1,21 grammes
2,37/2 le matin
3,99 francs
4,A history of violence
...,...
300,Les gardiens de la galaxie
301,Avengers
302,Yves saint laurent
303,Zero dark thirty


In [6]:
# Films de la liste d'Adrien à l'affiche en ce moment dans un cinéma parisien

films_merge = pd.merge(left=films_adrien, right=films_allociné, on='Title', how='left') # Ca suppose d'avoir exactement le même titre
films_merge.fillna(0,inplace=True)
films_en_commun = films_merge[films_merge['Code'] != 0]
films_en_commun['Code'] = films_en_commun['Code'].apply(lambda x: int(x))

/var/folders/yk/tz3m_phn5h10c9gz44v85x0r0000gn/T/ipykernel_26202/3626538239.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films_en_commun['Code'] = films_en_commun['Code'].apply(lambda x: int(x))


## Films de la liste à l'affiche au cinéma

> **La cellule en-dessous t'affiche tous les films de ta liste qui sont à l'affiche au cinéma**

In [7]:
# -- FILMS DE LA LISTE QUI SONT EN CE MOMENT A L'AFFICHE DANS UN CINEMA PARISIEN

if len(films_en_commun) == 0:
    print("Aucun film de la liste à l'affiche en ce moment dans un cinéma parisien")
elif len(films_en_commun) > 0:
    films_en_commun

In [13]:
films_en_commun

,Title,Code
188,Decision to leave,287116


# Informations sur les séances des films de ta liste

In [8]:
# Requete pour chaque film en commun

movies_code = list(films_en_commun['Code'].unique())

toutes_les_séances = list()

for code in movies_code:
    
    titre = films_allociné[films_allociné['Code'] == code].iloc[0,0]
    
    séances_film = {'Titre': titre, 'Cinéma': [], 'Adresse': [], 'Date&Version': [], 'Horaires': []}
    
    for d in range(0,7):
        url = f"https://www.allocine.fr/seance/film-{code}/pres-de-115767/d-{d}/"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        séances = soup.find_all('div', class_="theater-card hred cf")
        
        for séance in séances:
            nom_cinéma = str(séance.find('h2').find('a')).replace(str(séance.find('h2').find('a').attrs['href']),"").replace('href', "")
            nom_cinéma = re.sub("\"|\>|\<|\/|\=","",nom_cinéma)[1:-1].strip()
            séances_film["Cinéma"].append(nom_cinéma)

            adresse_cinéma = str(séance.find('address')).replace('address',"").replace('class',"")
            adresse_cinéma = re.sub("\"|\>|\<|\/|\=","",adresse_cinéma)[1:]
            séances_film["Adresse"].append(adresse_cinéma)
            
            info_séances = séance.find_all('div', class_="showtimes-version")[0]

            date_séance = str(info_séances.find_all('div', class_='text')).replace('class', "").replace('text',"").replace('div',"")
            date_séance = re.sub("\"|\>|\<|\/|\=|\n|\-","",date_séance)[1:-1]
            séances_film["Date&Version"].append(date_séance)

            horaire_séance = str(info_séances.find_all('span', class_='showtimes-hour-item-value'))#.replace('class', "").replace('showtimes-hour-item-value', "").replace('span', "")
            #horaire_séance = re.sub("\"|\>|\<|\/|\=|\n|\-","",horaire_séance)[1:-1]
            séances_film["Horaires"].append(horaire_séance)

    toutes_les_séances.append(séances_film)

In [14]:
toutes_les_séances

[{'Titre': 'Decision to leave',
  'Cinéma': [],
  'Adresse': [],
  'Date&Version': [],
  'Horaires': []}]

## Afficher les séances des films de la liste à l'affiche

> **La cellule en dessous affiche les informations sur les séances**

In [9]:
# -- INFORMATIONS SUR LES FILMS DE TA LISTE QUI SONT AU CINEMA

if len(toutes_les_séances) == 0:
    print("Aucun film de la liste à l'affiche en ce moment dans un cinéma parisien")
elif len(toutes_les_séances) > 0:
    for i in range(len(toutes_les_séances)):
        print(pd.DataFrame(toutes_les_séances[i])) 

Empty DataFrame
Columns: [Titre, Cinéma, Adresse, Date&Version, Horaires]
Index: []


# Pour avoir les séances d'un autre film sur AlloCiné

In [10]:
# Entrer le code du film recherché

code = 295626 # Code pour "Sur les chemins noirs" - modifier le code selon le film souhaité

In [11]:
# Exécuter la cellule

titre = films_allociné[films_allociné['Code'] == code].iloc[0,0]
séances_film = {'Ttire': titre, 'Cinéma': [], 'Adresse': [], 'Infos': []}

for d in range(0,7):
    
    url = f"https://www.allocine.fr/seance/film-{code}/pres-de-115767/d-{d}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    séances = soup.find_all('div', class_="theater-card hred cf")

    for séance in séances:
        nom_cinéma = séance.find('h2').find('a').string
        séances_film["Cinéma"].append(nom_cinéma)

        adresse_cinéma = séance.find('address').string
        séances_film["Adresse"].append(adresse_cinéma)

        info_séances = séance.find_all('div', class_="showtimes-version")
        
        séances_infos = {'Date&Version': [], 'Horaires': []}
        
        for info in info_séances:
            date_séance = str(info.find_all('div', class_='text')).replace('class', "").replace('text',"").replace('div',"")
            date_séance = re.sub("\"|\>|\<|\/|\=|\n|\-","",date_séance)[1:-1]
            séances_infos["Date&Version"].append(date_séance)

            horaire_séance = str(info.find_all('span', class_='showtimes-hour-item-value')).replace('class', "").replace('showtimes-hour-item-value', "").replace('span', "")
            horaire_séance = re.sub("\"|\>|\<|\/|\=|\n|\-","",horaire_séance)[1:-1]
            séances_infos["Horaires"].append(horaire_séance)
        
        séances_film['Infos'].append(séances_infos)

titre_séances = pd.DataFrame(séances_film)

## Exporter toutes les séances du film recherché

> **La cellule en dessous permet d'exporter les informations sur les séances du film choisi**

In [12]:
# Enlever le hastag et exécuter la cellule si besoin

#titre_séances.to_excel(f'Séances_{titre}.xlsx') 